# TensorFlow Training and Serving in SageMaker "Script Mode"

Script mode is a training script format for TensorFlow that lets you execute any TensorFlow training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native TensorFlow support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). In this example, we will show how easily you can train a SageMaker using TensorFlow 1.x and TensorFlow 2.x scripts with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker TensorFlow Serving container](https://github.com/aws/sagemaker-tensorflow-serving-container). The TensorFlow Serving container is the default inference method for script mode. For full documentation on the TensorFlow Serving container, please visit [here](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst).


# Set up the environment

Let's start by setting up the environment:

## Install TensorFlow and SageMaker
_Note:  Ignore Warnings and Errors Below_

In [1]:
!pip uninstall -y tensorflow tensorflow-estimator tb-nightly tf-estimator-nightly # tensorboard

Skipping tensorflow as it is not installed.
Skipping tensorflow-estimator as it is not installed.
Skipping tb-nightly as it is not installed.
Skipping tf-estimator-nightly as it is not installed.


In [2]:
!pip3 uninstall -y tensorflow tensorflow-estimator tb-nightly tf-estimator-nightly # tensorboard

Skipping tensorflow as it is not installed.
Skipping tensorflow-estimator as it is not installed.
Skipping tb-nightly as it is not installed.
Skipping tf-estimator-nightly as it is not installed.


In [3]:
!pip3 install tensorflow==2.0.0 --upgrade --ignore-installed --no-cache --user # tensorboard==1.14.0

    100% |████████████████████████████████| 86.3MB 69.3MB/s ta 0:00:011   38% |████████████▏                   | 32.8MB 67.3MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 43.0MB/s ta 0:00:01
    100% |████████████████████████████████| 2.4MB 96.0MB/s ta 0:00:011
    100% |████████████████████████████████| 51kB 53.9MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 77.1MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 51.5MB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 67.3MB/s ta 0:00:01
    100% |████████████████████████████████| 450kB 75.8MB/s ta 0:00:01
    100% |████████████████████████████████| 3.8MB 100.1MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 66.0MB/s ta 0:00:01
    100% |████████████████████████████████| 20.2MB 99.6MB/s ta 0:00:011
    100% |████████████████████████████████| 583kB 96.3MB/s ta 0:00:011
    100% |████████████████████████████████| 2.9MB 99.1MB/s ta 0:00:011
    100% |███████

    100% |████████████████████████████████| 163kB 84.6MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 99.3MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 70.6MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 103.1MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 68.6MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 97.9MB/s ta 0:00:01
coremltools 2.0 has requirement six==1.10.0, but you'll have six 1.13.0 which is incompatible.
botocore 1.12.66 has requirement urllib3<1.25,>=1.20; python_version >= "3.4", but you'll have urllib3 1.25.7 which is incompatible.
boto3 1.9.72 has requirement botocore<1.13.0,>=1.12.72, but you'll have botocore 1.12.66 which is incompatible.
awscli 1.16.76 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.0 which is incompatible.
  The scripts f2py, f2py3 and f2py3.6 are installed in '/home/ec2-user/.local/bin' which is not on PATH.
  Consider adding this directory to PA

In [4]:
!pip3 install sagemaker --upgrade --ignore-installed --no-cache --user

    100% |████████████████████████████████| 296kB 67.6MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 72.9MB/s ta 0:00:01
    100% |████████████████████████████████| 20.2MB 98.1MB/s ta 0:00:011
    100% |████████████████████████████████| 1.3MB 99.2MB/s ta 0:00:01
    100% |████████████████████████████████| 26.1MB 79.4MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 50.5MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 57.8MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 81.4MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 75.6MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 57.8MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 94.2MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 95.7MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 97.1MB/s ta 0:00:01
    100% |████████████████████████████████| 235kB 105.4MB/s ta 0:00:01
    100% |█████████

In [5]:
!pip3 install requests==2.20.1 --user

  Using cached https://files.pythonhosted.org/packages/ff/17/5cbb026005115301a8fb2f9b0e3e8d32313142fe8b617070e7baad20554f/requests-2.20.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4b/2a/0276479a4b3caeb8a8c1af2f8e4355746a97fab05a372e4a2c6a6b876165/idna-2.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/01/11/525b02e4acc0c747de8b6ccdab376331597c569c42ea66ab0a1dbd36eca2/urllib3-1.24.3-py2.py3-none-any.whl
tensorboard 2.0.2 has requirement requests<3,>=2.21.0, but you'll have requests 2.20.1 which is incompatible.
awscli 1.16.76 has requirement botocore==1.12.66, but you'll have botocore 1.13.49 which is incompatible.
awscli 1.16.76 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.0 which is incompatible.
awscli 1.16.76 has requirement s3transfer<0.2.0,>=0.1.12, but you'll have s3transfer 0.2.1 which is incompatible.
  Found existing installation: idna 2.8
    Uninstalling idna-2.8:
      Successfully uninstalled idna-

## Restart the Kernel to Recognize New Dependencies Above

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)

In [1]:
!pip3 list

Package              Version   
-------------------- ----------
absl-py              0.9.0     
astor                0.8.1     
astroid              1.6.6     
attrs                19.3.0    
awscli               1.16.76   
awscli-cwlogs        1.4.6     
backcall             0.1.0     
bleach               3.1.0     
bokeh                1.0.4     
boto                 2.49.0    
boto3                1.10.49   
botocore             1.13.49   
cachetools           4.0.0     
certifi              2019.11.28
chardet              3.0.4     
cloudpickle          1.2.2     
cmake                3.13.3    
colorama             0.3.9     
coremltools          2.0       
cpplint              1.3.0     
cycler               0.10.0    
dask                 2.6.0     
decorator            4.4.1     
defusedxml           0.6.0     
docutils             0.15.2    
entrypoints          0.3       
environment-kernels  1.1.1     
future               0.17.1    
gast                 0.2.2     
google-a

## Create the SageMaker Session

In [2]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

## Setup the Service Execution Role and Region
Get IAM role arn used to give training and hosting access to your data.  See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [3]:
role = get_execution_role()
print('RoleARN:  {}\n'.format(role))

region = sagemaker_session.boto_session.region_name
print('Region:  {}'.format(region))

RoleARN:  arn:aws:iam::362377691630:role/service-role/AmazonSageMaker-ExecutionRole-20200109T002600

Region:  us-east-1


## Training Data

The MNIST dataset has been loaded to the public S3 buckets ``sagemaker-sample-data-<REGION>`` under the prefix ``tensorflow/mnist``.

In [4]:
original_training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)
print(original_training_data_uri)

s3://sagemaker-sample-data-us-east-1/tensorflow/mnist


### Copy the Training Data to Your Notebook Disk

In [5]:
local_data_path = './data'

In [6]:
!aws --region {region} s3 cp --recursive {original_training_data_uri} {local_data_path}

download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/eval_labels.npy to data/eval_labels.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_labels.npy to data/train_labels.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/eval_data.npy to data/eval_data.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_data.npy to data/train_data.npy


There are four ``.npy`` file under this prefix:
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [7]:
!ls {local_data_path}

eval_data.npy  eval_labels.npy	train_data.npy	train_labels.npy


### Upload the Data to S3 for Distributed Training Across Many Workers
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

This is S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.

In [8]:
bucket = sagemaker_session.default_bucket()
data_prefix = 'sagemaker/tensorflow-mnist/data'

In [9]:
training_data_uri = sagemaker_session.upload_data(path=local_data_path, bucket=bucket, key_prefix=data_prefix)
print(training_data_uri)

s3://sagemaker-us-east-1-362377691630/sagemaker/tensorflow-mnist/data


In [10]:
!aws s3 ls --recursive {training_data_uri}

2020-01-09 05:39:38   31360128 sagemaker/tensorflow-mnist/data/eval_data.npy
2020-01-09 05:39:38      40128 sagemaker/tensorflow-mnist/data/eval_labels.npy
2020-01-09 05:39:38  172480128 sagemaker/tensorflow-mnist/data/train_data.npy
2020-01-09 05:39:38     220128 sagemaker/tensorflow-mnist/data/train_labels.npy


## Train
https://sagemaker.readthedocs.io/en/stable/using_tf.html#distributed-training

This tutorial's training script was adapted from TensorFlow's official [CNN MNIST example](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/layers/cnn_mnist.py). We have modified it to handle the ``model_dir`` parameter passed in by SageMaker. This is an S3 path which can be used for data sharing during distributed training and checkpointing and/or model persistence. We have also added an argument-parsing function to handle processing training-related variables.

At the end of the training job we have added a step to export the trained model to the path stored in the environment variable ``SM_MODEL_DIR``, which always points to ``/opt/ml/model``. This is critical because SageMaker uploads all the model artifacts in this folder to S3 at end of training.

### Training Script

In [11]:
!ls ./src/mnist_keras_tf2.py

./src/mnist_keras_tf2.py


You can add custom Python modules to the `src/requirements.txt` file.  They will automatically be installed - and made available to your training script.

In [12]:
!cat ./src/requirements.txt

# Python dependencies go here...

### Train with SageMaker `TensorFlow` Estimator

https://sagemaker.readthedocs.io/en/stable/using_tf.html#distributed-training

The `sagemaker.tensorflow.TensorFlow` estimator handles locating the script mode container, uploading your script to a S3 location and creating a SageMaker training job. Let's call out a couple important parameters here:

* `py_version` is set to `'py3'`
* `script_mode` is set to `True`
* `distributions` is used to configure the distributed training setup.  It's required only if you are doing distributed training either across a cluster of instances or across multiple GPUs.  Here we are using parameter servers as the distributed training schema. SageMaker training jobs run on homogeneous clusters.  To make parameter server more performant in the SageMaker setup, we run a parameter server on every instance in the cluster, so there is no need to specify the number of parameter servers to launch.  You can find the full documentation on how to configure `distributions` [here](https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#distributed-training).

Notes:  
* This example uses two(2) `ml.p3.2xlarge` instances.  You will likely need to request a SageMaker instance limit increase from Support before continuing.

* Alternatively, you can specify `ml.c5.2xlarge` or choose only one(1) `ml.p3.2xlarge`.

* To recognize the `requirements.txt`, we must include `src/setup.py` per [this](https://github.com/aws/sagemaker-python-sdk/issues/911) GitHub issue.

In [13]:
from sagemaker.tensorflow import TensorFlow

model_output_path = 's3://{}/sagemaker/tensorflow-mnist/training-runs'.format(bucket)

mnist_estimator = TensorFlow(entry_point='mnist_keras_tf2.py',
                             source_dir='./src',
                             output_path=model_output_path,
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.c5.2xlarge',
                             framework_version='2.0.0',
                             py_version='py3',
                             script_mode=True,
                             distributions={'parameter_server': {'enabled': True}},
                             tags = [{'Key' : 'Project', 'Value' : 'tensorflow-mnist'},
                                     {'Key' : 'TensorBoard', 'Value' : 'dist'}],
                             # Assuming the loglines from the TensorFlow training job are as follows:
                             #    Test loss    : 0.0635053280624561
                             #    Test accuracy: 0.9821
                             metric_definitions=[
                                 {'Name': 'test:loss', 'Regex': 'Test loss    : ([0-9\\.]+)'},
                                 {'Name': 'test:accuracy', 'Regex': 'Test accuracy: ([0-9\\.]+)'},
                             ])

### `fit` the Model (Approx. 15 mins)

To start a training job, we call `estimator.fit(training_data_uri)`.

An S3 location is used here as the input. `fit` creates a default channel named `'training'`, which points to this S3 location. In the training script we can then access the training data from the location stored in `SM_CHANNEL_TRAINING`. `fit` accepts other parameters, as well. See the API doc [here](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) for details.

In [14]:
mnist_estimator.fit(inputs=training_data_uri, wait=False)

training_job_name = mnist_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

training_job_name:  tensorflow-training-2020-01-09-05-39-43-463


After some time, or in a separate Python notebook, we can attach to the running job using the `training_job_name`.

In [ ]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow.attach(training_job_name=training_job_name)

2020-01-09 05:39:44 Starting - Starting the training job...
2020-01-09 05:40:14 Starting - Launching requested ML instances.........
2020-01-09 05:41:45 Starting - Preparing the instances for training.........
2020-01-09 05:42:59 Downloading - Downloading input data
2020-01-09 05:42:59 Training - Downloading the training image...
2020-01-09 05:43:38 Training - Training image download completed. Training in progress..2020-01-09 05:43:41,181 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-01-09 05:43:41,627 sagemaker-containers INFO     Installing module with the following command:
/usr/bin/python3 -m pip install . -r requirements.txt
Processing /opt/ml/code
  Created wheel for mnist-keras-tf2: filename=mnist_keras_tf2-1.0-cp36-none-any.whl size=1128 sha256=2bdbc374517adf0c2829ae1ae69e9cf5f6948ff9b721b8d0c02ed6e1a4ad10e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-trfvf4w6/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8c

In [ ]:
!aws --region {region} s3 ls --recursive {model_output_path}/{training_job_name}

In [ ]:
print(model_output_path)

## Option 1:  Predict Directly in the Notebook

Use TensorFlow Core to load the model from `model_output_path`

In [ ]:
!aws --region {region} s3 ls --recursive {model_output_path}/{training_job_name}/output/

In [ ]:
!aws --region {region} s3 cp {model_output_path}/{training_job_name}/output/model.tar.gz ./model/model.tar.gz

In [ ]:
saved_model_path = './model/000000001'

In [ ]:
!rm -rf {saved_model_path}

In [ ]:
!ls ./model

In [ ]:
!tar -xzvf ./model/model.tar.gz -C ./model

Load the model and list the prediction signatures

In [ ]:
import tensorflow as tf

loaded_model = tf.saved_model.load(export_dir='./model/000000001/')

loaded_model.signatures

Show the prediction signature details

In [ ]:
from tensorflow.python.tools import saved_model_cli

parser = saved_model_cli.create_parser()
args = parser.parse_args(['show', '--dir', saved_model_path, '--tag_set', 'serve', '--signature_def', 'serving_default'])
saved_model_cli.show(args)

### Perform inference with the signature

In [ ]:
import numpy as np

train_data = np.load('{}/train_data.npy'.format(local_data_path))
train_labels = np.load('{}/train_labels.npy'.format(local_data_path))

eval_data = np.load('{}/eval_data.npy'.format(local_data_path))
eval_labels = np.load('{}/eval_labels.npy'.format(local_data_path))

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

image_idx = 4444 # random idx number

plt.imshow(eval_data[image_idx].reshape(28, 28), cmap='Greys')

In [ ]:
predict_fn = loaded_model.signatures["serving_default"]

result = predict_fn(input_1=tf.constant(eval_data[image_idx].reshape(1, 28, 28, 1)))
prediction = result['output_1'].numpy().argmax()

label = eval_labels[image_idx]
print('Prediction is {}, Label is {}, Matched: {}'.format(prediction, label, prediction == label))

## Option 2:  Create a SageMaker Endpoint and Perform REST-based Predictions

### Deploy the Trained Model to a SageMaker Endpoint (Approx. 10 mins)

After training, we use the `TensorFlow` estimator object to build and deploy a `TensorFlowPredictor`.  This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `tensorflow_mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-tensorflow-containers](https://github.com/aws/sagemaker-tensorflow-containers).

The `deploy()` method creates a SageMaker model, which is then deployed to an endpoint to serve prediction requests in real time.  We will use the TensorFlow Serving container for the endpoint, because we trained with script mode.  This serving container runs an implementation of a web server that is compatible with SageMaker hosting protocol.  The [Using your own inference code](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) document explains how SageMaker runs inference containers.

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint.  These do not need to be the same as the values we used for the training job.  For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`.  Here we will deploy the model to a single `ml.p3.2xlarge` instance.  Alternatively, you can use a `ml.c5.2xlarge` instance.

In [ ]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.c5.2xlarge')

### Invoke the Endpoint

Let's download the training data and use that as input for inference.

In [ ]:
import numpy as np

train_data = np.load('{}/train_data.npy'.format(local_data_path))
train_labels = np.load('{}/train_labels.npy'.format(local_data_path))

The formats of the input and the output data correspond directly to the request and response formats of the `Predict` method in the [TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest). SageMaker's TensforFlow Serving endpoints can also accept additional input formats that are not part of the TensorFlow REST API, including the simplified JSON format, line-delimited JSON objects ("jsons" or "jsonlines"), and CSV data.

In this example we are using a `numpy` array as input, which will be serialized into the simplified JSON format. In addition, TensorFlow Serving can also process multiple items at once as you can see in the following code. You can find the complete documentation on how to make predictions against a TensorFlow Serving SageMaker endpoint [here](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint).

Examine the prediction result from the TensorFlow 2.0 model.

In [ ]:
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = predictions['predictions'][i]
    label = train_labels[i]
    print('Prediction is {}, Label is {}, Matched: {}'.format(prediction, label, prediction == label))

### (Optional) Cleanup Endpoint

Let's delete the endpoint we just created to prevent incurring any extra costs.

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)